# EdgesCGALWeightedDelaunay3D — Colab
Installe CGAL + TBB(+tbbmalloc), construit le binaire et exécute un test massif (1M points).


In [ ]:
%%bash
set -euxo pipefail
apt-get update -qq
DEBIAN_FRONTEND=noninteractive apt-get install -y -qq build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 libgmp-dev libmpfr-dev


In [ ]:
# Écrit le projet sur disque
import os, json
root = "/content/EdgesCGALWeightedDelaunay3D"
os.makedirs(root+"/src", exist_ok=True)
os.makedirs(root+"/data", exist_ok=True)
open(root+"/CMakeLists.txt","w").write("cmake_minimum_required(VERSION 3.22)\nproject(EdgesCGALWeightedDelaunay3D LANGUAGES CXX)\n\nset(CMAKE_CXX_STANDARD 17)\nset(CMAKE_CXX_STANDARD_REQUIRED ON)\nif(NOT CMAKE_BUILD_TYPE)\n  set(CMAKE_BUILD_TYPE Release CACHE STRING \"Build type\" FORCE)\nendif()\n\n# Silence Boost CMP0167 dev warning in newer CMake without changing user policies\nif(POLICY CMP0167)\n  cmake_policy(SET CMP0167 NEW)\nendif()\n\nfind_package(CGAL REQUIRED)\n# Try TBB; enable parallel mode only if we also find tbbmalloc\nfind_package(TBB QUIET)\n\nset(USE_TBB OFF)\nset(TBB_MALLOC_TARGET \"\")\nif(TBB_FOUND)\n  if(TARGET TBB::tbb)\n    set(USE_TBB ON)\n  endif()\n  if(TARGET TBB::tbbmalloc)\n    set(TBB_MALLOC_TARGET TBB::tbbmalloc)\n  else()\n    # Fallback: search the lib directly (Debian/Ubuntu often split it)\n    find_library(TBBMALLOC_LIB tbbmalloc)\n    if(TBBMALLOC_LIB)\n      set(TBB_MALLOC_TARGET ${TBBMALLOC_LIB})\n    endif()\n  endif()\n  if(USE_TBB AND TBB_MALLOC_TARGET STREQUAL \"\")\n    message(WARNING \"TBB found but tbbmalloc missing => disabling parallel CGAL path.\")\n    set(USE_TBB OFF)\n  endif()\nendif()\n\nadd_executable(EdgesCGALWeightedDelaunay3D src/EdgesCGALWeightedDelaunay3D.cpp)\ntarget_link_libraries(EdgesCGALWeightedDelaunay3D PRIVATE CGAL::CGAL)\n\nif(USE_TBB)\n  target_link_libraries(EdgesCGALWeightedDelaunay3D PRIVATE TBB::tbb ${TBB_MALLOC_TARGET})\n  target_compile_definitions(EdgesCGALWeightedDelaunay3D PRIVATE CGAL_LINKED_WITH_TBB)\nendif()\n\n# Speed knobs\nif(CMAKE_CXX_COMPILER_ID MATCHES \"GNU|Clang\")\n  target_compile_options(EdgesCGALWeightedDelaunay3D PRIVATE -O3 -DNDEBUG -march=native)\nelseif(MSVC)\n  target_compile_options(EdgesCGALWeightedDelaunay3D PRIVATE /O2 /DNDEBUG)\nendif()\n")
open(root+"/src/EdgesCGALWeightedDelaunay3D.cpp","w").write("// EdgesCGALWeightedDelaunay3D.cpp\n// Fast extraction of the 1-skeleton of the 3D Regular Triangulation (Weighted Delaunay) using CGAL.\n// Input: .xyzw with lines \"x y z w\" (w = CGAL power weight = squared radius).\n// Output: lines \"i j\" with 0-based indices of endpoints of each finite edge.\n#include <CGAL/Exact_predicates_inexact_constructions_kernel.h>\n#include <CGAL/Regular_triangulation_3.h>\n#include <CGAL/Regular_triangulation_vertex_base_3.h>\n#include <CGAL/Regular_triangulation_cell_base_3.h>\n#include <CGAL/Triangulation_vertex_base_with_info_3.h>\n#include <CGAL/Triangulation_data_structure_3.h>\n\n#include <cstdio>\n#include <cstdlib>\n#include <cstdint>\n#include <cstring>\n#include <vector>\n#include <string>\n#include <fstream>\n#include <iostream>\n#include <limits>\n#include <utility>\n#include <algorithm>\n#include <thread> // hardware_concurrency\n\n#ifdef CGAL_LINKED_WITH_TBB\n  #include <tbb/global_control.h>\n#endif\n\nstruct BBox {\n    double xmin =  std::numeric_limits<double>::infinity();\n    double ymin =  std::numeric_limits<double>::infinity();\n    double zmin =  std::numeric_limits<double>::infinity();\n    double xmax = -std::numeric_limits<double>::infinity();\n    double ymax = -std::numeric_limits<double>::infinity();\n    double zmax = -std::numeric_limits<double>::infinity();\n    void update(double x, double y, double z) {\n        xmin = std::min(xmin, x); ymin = std::min(ymin, y); zmin = std::min(zmin, z);\n        xmax = std::max(xmax, x); ymax = std::max(ymax, y); zmax = std::max(zmax, z);\n    }\n    bool valid() const { return xmin <= xmax && ymin <= ymax && zmin <= zmax; }\n};\n\nstatic inline bool parse_xyzw(const char* s, double& x, double& y, double& z, double& w) {\n    while (*s == ' ' || *s == '\\t') ++s;\n    if (*s == '\\0' || *s == '\\n' || *s == '\\r' || *s == '#') return false;\n    char* endp;\n    x = std::strtod(s, &endp); if (endp == s) return false; s = endp;\n    y = std::strtod(s, &endp); if (endp == s) return false; s = endp;\n    z = std::strtod(s, &endp); if (endp == s) return false; s = endp;\n    w = std::strtod(s, &endp); if (endp == s) return false;\n    return true;\n}\n\nint main(int argc, char** argv) {\n    if (argc != 3) {\n        std::fprintf(stderr,\n            \"Usage: %s input.xyzw output.edges\\n\"\n            \"Each input line: x y z w (floats). w is the CGAL power weight (squared radius).\\n\", argv[0]);\n        return 1;\n    }\n    const char* in_path = argv[1];\n    const char* out_path = argv[2];\n\n    using K   = CGAL::Exact_predicates_inexact_constructions_kernel;\n    using Vb0 = CGAL::Regular_triangulation_vertex_base_3<K>;\n    using Vb  = CGAL::Triangulation_vertex_base_with_info_3<uint32_t, K, Vb0>;\n    using Cb  = CGAL::Regular_triangulation_cell_base_3<K>;\n    #ifdef CGAL_LINKED_WITH_TBB\n      using Tds = CGAL::Triangulation_data_structure_3<Vb, Cb, CGAL::Parallel_tag>;\n    #else\n      using Tds = CGAL::Triangulation_data_structure_3<Vb, Cb>;\n    #endif\n    using Rt  = CGAL::Regular_triangulation_3<K, Tds>;\n    using Bare_point = Rt::Bare_point;\n    using Weighted_point = Rt::Weighted_point;\n\n    #ifdef CGAL_LINKED_WITH_TBB\n    {\n        int threads = (int)std::thread::hardware_concurrency();\n        if (const char* env = std::getenv(\"CGAL_NTHREADS\")) {\n            int req = std::atoi(env);\n            if (req > 0) threads = req;\n        }\n        static tbb::global_control gc(tbb::global_control::max_allowed_parallelism, threads);\n        std::fprintf(stderr, \"[info] TBB enabled, using up to %d threads\\n\", threads);\n    }\n    #endif\n\n    std::FILE* fin = std::fopen(in_path, \"rb\");\n    if (!fin) { std::perror(\"fopen(input)\"); return 2; }\n\n    std::vector<std::pair<Weighted_point, uint32_t>> pts;\n    pts.reserve(1<<20);\n    BBox bb;\n\n    // bulk read\n    {\n        const size_t BUFSZ = size_t(1) << 20;\n        std::vector<char> buf(BUFSZ);\n        std::string acc; acc.reserve(BUFSZ);\n        while (true) {\n            size_t n = std::fread(buf.data(), 1, BUFSZ, fin);\n            if (n == 0) break;\n            for (size_t i = 0; i < n; ++i) {\n                char c = buf[i];\n                if (c == '\\r') continue;\n                if (c == '\\n') {\n                    double x,y,z,w;\n                    acc.push_back('\\0');\n                    if (parse_xyzw(acc.c_str(), x,y,z,w)) {\n                        if (std::isfinite(x) && std::isfinite(y) && std::isfinite(z) && std::isfinite(w)) {\n                            bb.update(x,y,z);\n                            pts.emplace_back(Weighted_point(Bare_point(x,y,z), K::FT(w)),\n                                             (uint32_t)pts.size());\n                        }\n                    }\n                    acc.clear();\n                } else acc.push_back(c);\n            }\n        }\n        if (!acc.empty()) {\n            double x,y,z,w;\n            acc.push_back('\\0');\n            if (parse_xyzw(acc.c_str(), x,y,z,w)) {\n                if (std::isfinite(x) && std::isfinite(y) && std::isfinite(z) && std::isfinite(w)) {\n                    bb.update(x,y,z);\n                    pts.emplace_back(Weighted_point(Bare_point(x,y,z), K::FT(w)),\n                                     (uint32_t)pts.size());\n                }\n            }\n        }\n    }\n    std::fclose(fin);\n\n    if (pts.size() < 2) {\n        std::fprintf(stderr, \"Input has < 2 valid points. Nothing to do.\\n\");\n        std::FILE* fout = std::fopen(out_path, \"wb\");\n        if (fout) std::fclose(fout);\n        return 0;\n    }\n    std::fprintf(stderr, \"[info] Loaded %zu points\\n\", pts.size());\n\n    // Triangulation\n    #ifdef CGAL_LINKED_WITH_TBB\n    std::unique_ptr<Rt::Lock_data_structure> lock_holder;\n    Rt::Lock_data_structure* lock_ptr = nullptr;\n    if (bb.valid()) {\n        CGAL::Bbox_3 box(bb.xmin, bb.ymin, bb.zmin, bb.xmax, bb.ymax, bb.zmax);\n        lock_holder.reset(new Rt::Lock_data_structure(box, 64));\n        lock_ptr = lock_holder.get();\n    }\n    Rt rt(Rt::Geom_traits(), lock_ptr);\n    #else\n    Rt rt;\n    #endif\n\n    rt.insert(pts.begin(), pts.end());\n    if (!rt.is_valid(false))\n        std::fprintf(stderr, \"[warning] CGAL triangulation reports invalid structure.\\n\");\n\n    // Write edges\n    std::FILE* fout = std::fopen(out_path, \"wb\");\n    if (!fout) { std::perror(\"fopen(output)\"); return 3; }\n    const size_t OUTBUFSZ = size_t(1) << 20;\n    std::vector<char> filebuf(OUTBUFSZ);\n    setvbuf(fout, filebuf.data(), _IOFBF, filebuf.size());\n\n    std::vector<char> outbuf;\n    outbuf.reserve(OUTBUFSZ);\n    auto flush_buf = [&](){ if (!outbuf.empty()) { std::fwrite(outbuf.data(),1,outbuf.size(),fout); outbuf.clear(); } };\n\n    size_t edge_count = 0;\n    for (auto eit = rt.finite_edges_begin(); eit != rt.finite_edges_end(); ++eit) {\n        auto c = eit->first;\n        int i = eit->second, j = eit->third;\n        auto vi = c->vertex(i);\n        auto vj = c->vertex(j);\n        if (rt.is_infinite(vi) || rt.is_infinite(vj)) continue;\n        uint32_t a = vi->info(), b = vj->info();\n        if (a == b) continue;\n        if (a > b) std::swap(a,b);\n        char line[64];\n        int len = std::snprintf(line, sizeof(line), \"%u %u\\n\", a, b);\n        if (outbuf.size() + (size_t)len > OUTBUFSZ) flush_buf();\n        outbuf.insert(outbuf.end(), line, line + len);\n        ++edge_count;\n    }\n    flush_buf();\n    std::fclose(fout);\n    std::fprintf(stderr, \"[info] Wrote %zu edges to %s\\n\", edge_count, out_path);\n    return 0;\n}\n")
open(root+"/README.md","w").write("# EdgesCGALWeightedDelaunay3D\n\nExtraction rapide du **1\u2011squelette** de la triangulation r\u00e9guli\u00e8re 3D (Delaunay pond\u00e9r\u00e9) avec **CGAL**.\n\n- **Entr\u00e9e**: `.xyzw` o\u00f9 chaque ligne contient `x y z w` (float). `w` est le **poids de puissance** de CGAL (si vous pensez en rayon `r`, utilisez `w = r^2`).\n- **Sortie**: texte, une ligne par ar\u00eate finie: `i j` (indices 0\u2011based des points).\n\n## D\u00e9pendances (Ubuntu 22.04/24.04)\n```bash\nsudo apt-get update\nsudo apt-get install -y build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 libgmp-dev libmpfr-dev\n```\n> `tbbmalloc` est explicitement link\u00e9 pour \u00e9viter les symboles non r\u00e9solus `scalable_malloc/free`.\n\n## Build\n```bash\ncmake -S . -B build -DCMAKE_BUILD_TYPE=Release\ncmake --build build -j\n```\n\n## Utilisation\n```bash\n./build/EdgesCGALWeightedDelaunay3D data/example.xyzw out.edges\n# Parall\u00e9lisme (si TBB trouv\u00e9 + tbbmalloc pr\u00e9sent) :\nCGAL_NTHREADS=$(nproc) ./build/EdgesCGALWeightedDelaunay3D data/example.xyzw out.edges\n```\n\n## Donn\u00e9es de test\n`data/example.xyzw` contient 1000 points uniformes dans `[0,1]^3` avec des poids plausibles (`w=r^2`, `r~U(0,0.02)`).\n\n## Remarques\n- Si `tbb` est trouv\u00e9 **sans** `tbbmalloc`, le build **d\u00e9sactive** la voie parall\u00e8le pour \u00e9viter les erreurs de linkage (`scalable_*`).  \n- Insertion **par lot** des points avec `vertex->info()` renseign\u00e9 (indice d'origine), puis it\u00e9ration sur `finite_edges_*`.\n")
print("Projet écrit dans", root)


In [ ]:
%%bash
set -euxo pipefail
cd /content/EdgesCGALWeightedDelaunay3D
cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
cmake --build build -j
ls -lah build/EdgesCGALWeightedDelaunay3D


In [ ]:
# Génère N points uniformes + poids plausibles
import numpy as np
N = 1_000_000
rng = np.random.default_rng(0)
path = "/content/EdgesCGALWeightedDelaunay3D/data/test.xyzw"
with open(path, 'w') as f:
    CHUNK = 200_000
    remain = N
    while remain>0:
        m = min(CHUNK, remain)
        xyz = rng.random((m,3))
        r = 0.02 * rng.random(m)
        w = r*r
        for i in range(m):
            f.write(f"{xyz[i,0]} {xyz[i,1]} {xyz[i,2]} {w[i]}\n")
        remain -= m
print('Écrit', N, 'points dans', path)


In [ ]:
%%bash
set -euxo pipefail
cd /content/EdgesCGALWeightedDelaunay3D
CGAL_NTHREADS=$(nproc)
time ./build/EdgesCGALWeightedDelaunay3D data/test.xyzw out.edges
echo "Nombre d'arêtes:" $(wc -l < out.edges)
head -n 10 out.edges
